In [12]:
import numpy as np 
import pandas as pd 
from functools import reduce 
import time

In [8]:
files = ['HDFCBANK.csv','ITC.csv','LT.csv','M&M.csv','TCS.csv']
dfs = []

for file in files : 
    temp = pd.read_csv('./data/'+file)
    temp.columns = ['Date',file.replace('.csv','')]
    dfs.append(temp)

stocks = reduce(lambda left ,right : pd.merge(left,right,on='Date'),dfs)
print(stocks.shape)
stocks.head()

(411, 6)


,Date,HDFCBANK,ITC,LT,M&M,TCS
0,2022-01-03,1519.650024,219.100006,1922.849976,829.799988,3817.750000
1,2022-01-04,1528.550049,220.250000,1937.550049,831.849976,3884.750000
2,2022-01-05,1564.849976,220.550003,1948.599976,839.500000,3860.949951
3,2022-01-06,1539.750000,218.699997,1924.500000,839.849976,3807.449951
4,2022-01-07,1550.550049,218.399994,1904.900024,829.000000,3853.500000


In [9]:
def hist_return(months): 
    idx = []
    df = pd.DataFrame()
    for mon in months : 
        temp =(stocks.iloc[0,1:] - stocks.iloc[mon,1:])/(stocks.iloc[mon,1:])
        idx.append(str(mon)+"_mon_return")
        df = pd.concat([df,temp.to_frame().T] ,ignore_index=True)
    df.index = idx
    return df 

In [10]:
hist_stocks_returns = hist_return([3,6,12,24,36])
hist_stocks_returns

,HDFCBANK,ITC,LT,M&M,TCS
3_mon_return,-0.013054,0.001829,-0.000857,-0.011966,0.002705
6_mon_return,-0.029536,-0.012841,-0.019529,-0.013787,-0.025064
12_mon_return,0.00079,-0.005673,-0.048518,-0.072227,-0.024753
24_mon_return,0.035078,-0.048219,0.02035,0.000362,0.010254
36_mon_return,0.012493,0.014587,0.056337,-0.024109,0.071258


In [7]:
pd.read_csv('./data/HDFCBANK.csv')

,Date,Adj Close
0,2022-01-03,1485.152466
1,2022-01-04,1493.850464
2,2022-01-05,1529.326416
3,2022-01-06,1504.796143
4,2022-01-07,1515.351074
...,...,...
406,2023-08-23,1586.599976
407,2023-08-24,1579.300049
408,2023-08-25,1561.500000
409,2023-08-28,1577.750000


In [11]:
gene = np.random.rand()
gene

0.5653856826001153

In [13]:
def gene_mc_grid(rows,cols,n,N): 
    np.random.seed(seed=int(time.time()))
    layouts = np.zeros((n,rows*cols),dtype=np.int32)
    positionX = np.random.randint(0 ,cols , size=(N * n * 2))
    positionY = np.random.randint(0 ,rows, size=(N *n * 2))
    ind_rows = 0 
    ind_pos = 0

    while ind_rows < n : 
        layouts[ind_rows , positionX[ind_pos] + positionY[ind_pos] * cols] = 1 
        if np.sum(layouts[ind_rows , :]) == N : 
            ind_rows += 1 
        ind_pos += 1 
        if ind_pos >= N * n * 2 : 
            print("Not enough postiton")
            break 

    return layouts


def gene_mc_grid_with_NA_loc(rows,cols,n,N,NA_loc): 
    np.random.seed(seed=int(time.time()))
    layouts = np.zeros((n,rows*cols),dtype=np.int32)
    layouts_NA = np.zeros((n , rows*cols), dtype=np.int32)
    for i in NA_loc : 
        layouts_NA[: ,i-1]=2 

    positionX = np.random.randint(0 ,cols , size=(N * n * 2))
    positionY = np.random.randint(0 ,rows, size=(N *n * 2))
    ind_rows = 0 
    ind_pos = 0

    N_count = 0 
    while ind_rows < n : 
        cur_state = layouts_NA[ind_rows,positionX[ind_pos] +    positionY[ind_pos]*cols]
        if cur_state != 1 and cur_state !=2 : 
            layouts[ind_rows,positionX[ind_pos] + positionY[ind_pos]*cols]=1 
            layouts_NA[ind_rows,positionX[ind_pos] + positionY[ind_pos]*cols]=1
            N_count += 1 
        
            if np.sum(layouts[ind_rows , :]) == N : 
                ind_rows += 1 
                N_count = 0 
        ind_pos += 1 
        if ind_pos >= N * n * 2 : 
            print("Not enough positions")
            break 
    
    return layouts , layouts_NA 
        

In [14]:
gene_mc_grid(5, 5, 100, 50)
gene_mc_grid_with_NA_loc(5, 5, 100, 50,range(10))

Not enough postiton
Not enough positions


(array([[0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]),
 array([[2, 2, 2, ..., 1, 1, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        ...,
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2],
        [2, 2, 2, ..., 0, 0, 2]]))

In [17]:
def chromosome(n): 
    ch = np.random.rand(n)
    return ch / sum(ch)

child = chromosome(5)
print(child , sum(child))

[0.10682904 0.09328513 0.05295805 0.36409863 0.38282914] 1.0
